In [ ]:
import numpy as np
import pandas as pd
import os
import os.path as path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, SimpleRNN,GRU, Flatten, Dropout,GlobalMaxPooling1D,Conv1D, Bidirectional
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.regularizers import l2
import re
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
#os.chdir("/Users/nannakildahlmathiasen/OneDrive/8_semester/Data_science/ExamProject/bibleproject2")
#predictors = pd.read_csv("web_bible.csv", sep = ",")
# Upload the train file from your local drive
from google.colab import files
uploaded = files.upload()

Saving web_bible.csv to web_bible.csv


In [ ]:
df = pd.read_csv("web_bible.csv", delimiter=',', header=None,names=['id', 'b', 'c', 'v','t'])

In [ ]:
df.head()

,id,b,c,v,t
0,id,b,c,v,t
1,1001001,1,1,1,"In the beginning God{After ""God,"" the Hebrew h..."
2,1001002,1,1,2,Now the earth was formless and empty. Darkness...
3,1001003,1,1,3,"God said, ""Let there be light,"" and there was ..."
4,1001004,1,1,4,"God saw the light, and saw that it was good. G..."


Dividing into Old and New Testament

In [ ]:
df.loc[0:23144,'label'] = 0
df.loc[23145:,'label'] = 1
df['t'] = df.t.str.replace('[\.\,\'\"\?\:\;\-\_\=\(\)\|\*\@\#\&\$\"\/\%\+]', '')
df = df[['t', 'label']].copy()
df['label'] = df['label'].apply(np.float32)

In [ ]:
# in order to save the new csv
from google.colab import drive
drive.mount('drive')

In [ ]:
df.to_csv('GTNT.csv')
!cp GTNT.csv "drive/My Drive/"

Making a balanced sample for fine-tuning to be faster

In [ ]:
# sample a smaller, balanced dataset to investigate hyperparameter 
s0 = df.label[df.label.eq(0)].sample(5000).index
s1 = df.label[df.label.eq(1)].sample(5000).index 

df = df.loc[s0.union(s1)]

In [ ]:
df.to_csv('GTNT_training.csv')
!cp GTNT_training.csv "drive/My Drive/"

Making a subset with only the gospels

In [ ]:
# Upload the train file from your local drive
from google.colab import files
uploaded = files.upload()

Saving web_bible.csv to web_bible.csv


In [ ]:
df = pd.read_csv("web_bible.csv", delimiter=',', header=None,names=['id', 'b', 'c', 'v','t'])

In [ ]:
df = df[['t', 'b']].copy()

In [ ]:
df = df.loc[df['b'].isin(['40','41','42','43'])]

In [ ]:
print(df)

                                                       t   b
23146  The book of the generation of Jesus Christ{Chr...  40
23147  Abraham became the father of Isaac. Isaac beca...  40
23148  Judah became the father of Perez and Zerah by ...  40
23149  Ram became the father of Amminadab. Amminadab ...  40
23150  Salmon became the father of Boaz by Rahab. Boa...  40
...                                                  ...  ..
26920  Peter seeing him, said to Jesus, "Lord, what a...  43
26921  Jesus said to him, "If I desire that he stay u...  43
26922  This saying therefore went out among the broth...  43
26923  This is the disciple who testifies about these...  43
26924  There are also many other things which Jesus d...  43

[3779 rows x 2 columns]


In [ ]:
df.columns = ["t","label"]

In [ ]:
df['t'] = df.t.str.replace('[\.\,\'\"\?\:\;\-\_\=\(\)\|\*\@\#\&\$\"\/\%\+\{\}]', '')

In [ ]:
print(df.t.str.len().max())
print(df.t.str.len().min())
print(df.t.str.len().mean())

569
10
108.18629267001852


In [ ]:
df['label'] = df.label.map({'40': 0, '41': 1,'42': 2,'43': 3})

Saving the file with only the gospels

In [ ]:
df.to_csv('all_gospels.csv')
!cp all_gospels.csv "drive/My Drive/"

Making a balanced sample for fine-tuning

In [ ]:
df.head()
s0 = df.label[df.label.eq(0)].sample(600).index
s1 = df.label[df.label.eq(1)].sample(600).index 
s2 = df.label[df.label.eq(2)].sample(600).index
s3 = df.label[df.label.eq(3)].sample(600).index 

#df = df.loc[s0.union(s1)]

In [ ]:
df1 = df.loc[s0.union(s1)]

In [ ]:
df2=df.loc[s2.union(s3)]

In [ ]:
result = df1.append(df2)

In [ ]:
print(result)

                                                       t  label
23149  Ram became the father of Amminadab Amminadab b...      0
23150  Salmon became the father of Boaz by Rahab Boaz...      0
23151  Jesse became the father of David the king Davi...      0
23153  Asa became the father of Jehoshaphat Jehoshaph...      0
23155  Hezekiah became the father of Manasseh Manasse...      0
...                                                  ...    ...
26919  Then Peter turning around saw a disciple follo...      3
26920  Peter seeing him said to Jesus Lord what about...      3
26921  Jesus said to him If I desire that he stay unt...      3
26922  This saying therefore went out among the broth...      3
26923  This is the disciple who testifies about these...      3

[2400 rows x 2 columns]


In [ ]:
result.to_csv('training_gospels.csv')
!cp training_gospels.csv "drive/My Drive/"